In [1]:
import ezdxf
import xml.etree.ElementTree as ET
import math
import numpy as np

def dxf_to_xml(dxf_file, xml_file, num_points=20):
    doc = ezdxf.readfile(dxf_file)
    msp = doc.modelspace()

    shape = ET.Element("shape")
    waypoints = ET.SubElement(shape, "waypoints")

    index = 1

    # 圆
    for circle in msp.query("CIRCLE"):
        center = circle.dxf.center
        radius = circle.dxf.radius
        for i in range(num_points):
            theta = 2 * math.pi * i / num_points
            x = center.x + radius * math.cos(theta)
            y = center.y + radius * math.sin(theta)
            z = center.z if center.z else 0
            ET.SubElement(waypoints, "waypoint",
                          index=str(index), x=str(x), y=str(y), z=str(z),
                          r="0", speed="5")
            index += 1

    # 直线
    for line in msp.query("LINE"):
        start, end = line.dxf.start, line.dxf.end
        xs = np.linspace(start.x, end.x, num_points)
        ys = np.linspace(start.y, end.y, num_points)
        zs = np.linspace(start.z or 0, end.z or 0, num_points)
        for x, y, z in zip(xs, ys, zs):
            ET.SubElement(waypoints, "waypoint",
                          index=str(index), x=str(x), y=str(y), z=str(z),
                          r="0", speed="8")
            index += 1

    # 折线/矩形
    for pl in msp.query("LWPOLYLINE POLYLINE"):
        vertices = list(pl.get_points("xy"))
        for i in range(len(vertices) - (0 if pl.closed else 1)):
            x1, y1 = vertices[i]
            x2, y2 = vertices[(i+1) % len(vertices)]
            xs = np.linspace(x1, x2, num_points)
            ys = np.linspace(y1, y2, num_points)
            for x, y in zip(xs, ys):
                ET.SubElement(waypoints, "waypoint",
                              index=str(index), x=str(x), y=str(y), z="0",
                              r="0", speed="8")
                index += 1

    # 圆弧
    for arc in msp.query("ARC"):
        center = arc.dxf.center
        radius = arc.dxf.radius
        start_angle = math.radians(arc.dxf.start_angle)
        end_angle = math.radians(arc.dxf.end_angle)
        for i in range(num_points):
            theta = start_angle + (end_angle - start_angle) * i / (num_points - 1)
            x = center.x + radius * math.cos(theta)
            y = center.y + radius * math.sin(theta)
            z = center.z if center.z else 0
            ET.SubElement(waypoints, "waypoint",
                          index=str(index), x=str(x), y=str(y), z=str(z),
                          r="0", speed="8")
            index += 1

    # 样条曲线
    for spline in msp.query("SPLINE"):
        bspline = spline.construction_tool()
        points = [(p.x, p.y, p.z) for p in bspline.approximate(num_points)]
        for x, y, z in points:
            ET.SubElement(waypoints, "waypoint",
                          index=str(index), x=str(x), y=str(y), z=str(z),
                          r="0", speed="8")
            index += 1

    # 美化输出（Python 3.9+）
    tree = ET.ElementTree(shape)
    ET.indent(tree, space="    ")
    tree.write(xml_file, encoding="utf-8", xml_declaration=True)
    print(f"已导出XML文件: {xml_file}")

dxf_to_xml("shapes.dxf", "shapes.xml", num_points=20)


已导出XML文件: shapes.xml
